In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime
import locale

In [2]:
df = pd.read_csv("dataset/movies_dataset.csv",low_memory=False)

In [3]:

def fetch_name(obj): 
    if isinstance(obj, str) and '{' in obj:
        L = []
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L

In [4]:
def fetch_name_2(obj): 
    if isinstance(obj, str) and '{' in obj:
        dic = ast.literal_eval(obj)
        return dic['name']

In [5]:
df["genres"] = df["genres"].apply(fetch_name)
df["belongs_to_collection"] = df["belongs_to_collection"].apply(fetch_name_2)
df["production_companies"]  = df["production_companies"].apply(fetch_name)
df["production_countries"]  = df["production_countries"].apply(fetch_name)
df["spoken_languages"] = df["spoken_languages"].apply(fetch_name)

In [6]:
fechas_datetime = pd.to_datetime(df["release_date"], format='%Y-%m-%d', errors = 'coerce')
df ['release_date'] = fechas_datetime.dropna()


In [7]:
quitar_null = df[df["release_date"].isnull()]
df = df.drop(labels= quitar_null.index, axis=0)

In [8]:

df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")
df["return"] = df["revenue"] / df["budget"]
df["return"] = df["return"].replace([np.inf, -np.inf], np.nan)
df["return"] = df["return"].fillna(0)
df["return"] = df["return"].round(2)


In [9]:
df["revenue"].fillna("0",inplace=True)
df["budget"].fillna("0",inplace=True)

In [10]:
df.drop("video",axis="columns",inplace=True)
df.drop("imdb_id",axis="columns",inplace=True)
df.drop("adult",axis="columns",inplace=True)
df.drop("original_title",axis="columns",inplace=True)
df.drop("vote_count",axis="columns",inplace=True)
df.drop("poster_path",axis="columns",inplace=True)
df.drop("homepage",axis="columns",inplace=True)

In [11]:
df["month_name"] = df["release_date"].dt.month_name(locale.setlocale(locale.LC_TIME, 'es_ES'))
df["day_name"] = df["release_date"].dt.day_name(locale.setlocale(locale.LC_TIME, 'es_ES'))

In [12]:
def peliculas_mes(mes):
    aux = df.groupby("month_name").size()
    respuesta = aux [mes]

    return {'mes':mes, 'cantidad':respuesta}

In [13]:
def peliculas_dia(dia):
    aux = df.groupby("day_name").size()
    respuesta = aux [dia]
    return {'dia':dia, 'cantidad':respuesta}

In [14]:
def franquicia(franquicia):
    aux = df.groupby("belongs_to_collection").size()
    cantidad = aux [franquicia]
    ganancia = df.query("belongs_to_collection == @franquicia")['revenue'].sum()
    promedio = df.query("belongs_to_collection == @franquicia")['revenue'].mean()
    return {'franquicia':franquicia, 'cantidad de peliculas':cantidad, 'ganancia_total':ganancia, 'ganancia_promedio':promedio}

In [15]:
def peliculas_pais(pais):
    df["production_countries"] = df["production_countries"].fillna("")
    df_filtrado = df[df['production_countries'].apply(lambda x: pais in str(x))]
    cantidad_peliculas = len(df_filtrado)
    return {'pais':pais, 'cantidad':cantidad_peliculas}

In [16]:
def productoras(productora):
    df_filtrado = df[df['production_companies'].apply(lambda x: productora in str(x))]
    cantidad = len(df_filtrado)
    ganancia= df_filtrado["revenue"].sum()
    return {'productora':productora, 'ganancia_total':ganancia, 'cantidad':cantidad}

In [17]:
def retorno(pelicula):
    df_filtrado = df[df['title'].apply(lambda x: pelicula.strip() == str(x).strip())]
    inversion= df_filtrado["budget"].sum()
    ganancia= df_filtrado["revenue"].sum()
    retorno= df_filtrado["return"].sum()
    anio = df_filtrado["release_date"]
    return {'pelicula':pelicula, 'inversion':inversion, 'ganacia':ganancia,'retorno':retorno, 'anio':anio}

In [18]:
# df.to_csv("dataset_cleaned.csv", index=False)

In [19]:
df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,return,month_name,day_name
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,12.45,Octubre,Lunes
1,None,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,4.04,Diciembre,Viernes
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,0.00,Diciembre,Viernes
3,None,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,5.09,Diciembre,Viernes
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,0.00,Febrero,Viernes
